In [6]:
%load_ext autoreload
%autoreload 2

SyntaxError: invalid syntax (<string>, line 1)

# Effecient Multi-Party Private Set Intersection

### By Malia Kency and John Owens

First, load imports

In [7]:
import math
import protocol as P
import bloom_filter as BF

Set constant protocol parameters:

   * $NumPlayers$ = Total number of players $P_i$
   * $PlayerInputSize$ = Size of the player's input sets
   * $N_{BF}$ = Length of the Bloom Filter
   * $k$ = Number of hash functions to use
   * $\kappa = SecParam$ = Security paremeter. 40 in the paper's example
   * $N_{Maxones}$ = Maximum number of ones a player is allowed after cut-and-choose
   * $p$ = percentage of total messages to be used for cut-and-choose
   * $a$ = sampling weight of 1s vs. 0s for every $P_i$

Then calculate the rest:
   * $N_{OT}$ = Total number of Random Oblivious Transfers
   * $m_h^1$ = The number of 1's a player chooses
   * $\gamma = gamma$ = Verifies the correct relationship between $p, \kappa, m_h^1$
   * $\gamma^* = gammaStar$ = Verifies the correct relationship between $p, \kappa, N_{OT}$

In [10]:
NumPlayers = 3 
PlayerInputSize = 5
Nbf = 10
k = BF.GetNumHashFuncs(Nbf, PlayerInputSize)
SecParam = 20
Nmaxones = 10
p = 0.25
a = 0.3

# Initialize the protocol with the parameters above
Protocol = P.Protocol(Nmaxones, PlayerInputSize, k, p, a, SecParam, Nbf)

# Add the players to the protocol
Protocol.AddPlayers(NumPlayers)

Player 0 created
Player 1 created
Player 2 created


After parameters are set, we can simluate the first major step of the protocol: Random OT

Note: The number of Oblivious Transfers that occur is determined by $N_{OT}$, which is the result of several calculations in `security_parameters.py`

In [9]:
Protocol.Perform_RandomOT(Protocol.Params.a, Protocol.Params.Not)
print("Player 0 has " + str(Protocol.players[0].getNumberOfOnes()) + " ones. Should be: " + str(Protocol.Params.Not * (NumPlayers - 1)))
print("Player 1 has " + str(Protocol.players[1].getNumberOfOnes()) + " ones. Ideal: " + str(Protocol.Params.Not * Protocol.Params.a))
print("Player 2 has " + str(Protocol.players[2].getNumberOfOnes()) + " ones. Ideal: " + str(Protocol.Params.Not * Protocol.Params.a))
print("\'a\' defined as " + str(Protocol.Params.a));

Performing 67 Random Oblivious Transfers...
Player 0 has 134 ones. Should be: 134
Player 1 has 28 ones. Ideal: 20.099999999999998
Player 2 has 24 ones. Ideal: 20.099999999999998
'a' defined as 0.3
